In [178]:
"""
this notebook studies how asset possession varies with income
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys; sys.path.append('/home/mitch/school/data/mexico_enigh/src/')

import utils
import os

import statsmodels.stats.weightstats as ws

plotdir = '/home/mitch/school/data/mexico_enigh/fig/'

processed = '/home/mitch/Dropbox/data/mexico_enigh/processed/'
spec = '/home/mitch/Dropbox/data/mexico_enigh/spec/1996/'

In [179]:
os.chdir(processed)
perceptions = pd.read_csv('capital_income.csv')
expenditures = pd.read_csv('capital_expenditures.csv')

df = pd.read_csv('data.csv')

/tmp/ipykernel_99174/757368510.py:5: DtypeWarning: Columns (92,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [180]:
years = [1994, 1996, 1998, 2000]
df = df.query('Y in @years')

In [181]:
df = (df.merge(perceptions, how='outer', on='hhid')
      .merge(expenditures,how='outer', on='hhid'))
df = df[~np.isnan(df.location)]
df

,hhid,location,weight,location_size,ingmon,permon,gasnom,eronom,income_current,gasmon,...,expense_hhbusiness_negative_balance,expense_life_insurance,expense_loans_to_third_parties,expense_other,expense_realestate_hh_inhabit,expense_realestate_hh_not_inhabit,expense_realestate_mortgage,expense_securities_shares_bond,expense_trademarks_patents_copyrights,expense_inheritance_dowrie_legacies
0,19940110010,1001.0,169.0,1.0,6480.00,0.0,920.31,1680.0,7400.31,5062.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19940110030,1001.0,169.0,1.0,2760.00,0.0,593.54,0.0,3353.54,2002.79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19940110040,1001.0,169.0,1.0,6600.00,0.0,1533.85,0.0,8133.85,4925.66,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,19940110060,1001.0,153.0,1.0,7560.00,200.0,1305.79,0.0,8865.79,4109.89,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,19940110070,1001.0,153.0,1.0,13000.00,500.0,3185.04,0.0,16185.04,11161.73,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36551,200032190190,32042.0,1392.0,4.0,7874.42,0.0,1923.45,0.0,9797.87,5216.13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
36552,200032190200,32042.0,1392.0,4.0,11876.99,0.0,3352.27,0.0,15229.26,10818.09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
36553,200032190210,32042.0,1392.0,4.0,15285.00,5000.0,12034.06,0.0,27319.06,8041.58,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
36554,200032190220,32042.0,1392.0,4.0,15592.49,2500.0,3375.53,0.0,18968.02,9584.18,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN


In [182]:
income = ['income_salary', 'income_business', 'income_transfers']
df['income'] = df[income].sum(axis=1)
df['log_income'] = np.log(df['income'])

consumption = ['consumption_food', 'consumption_personal']
#consumption = ['consumption_food', 'consumption_clothing', 'consumption_personal', 'consumption_education']
df['consumption'] = df[consumption].sum(axis=1)
df['log_consumption'] = np.log(df['consumption'])

/home/mitch/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/mitch/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [183]:
df = df.query('consumption > 0')
df = df.query('income > 0')
df['income'] = df['income'] / df['hh_size']
len(df)

36371

In [184]:
perceptions.columns

Index(['hhid', 'income_coins_preciousmetals_art',
       'income_from_loans_to_institutions_not_mortgage',
       'income_from_loans_to_third_parties_not_in_household',
       'income_hhbusiness_machinery_animal_equipment',
       'income_house_land_condiminum', 'income_interest_fixed_investments',
       'income_interest_loans_to_third_parties',
       'income_interest_savings_accounts', 'income_interest_shares_bonds',
       'income_land_landrental', 'income_realestate_mortgage',
       'income_rental_houses_buildings_other_realestate',
       'income_rental_trademarks_patents_copyrights',
       'income_securities_bonds', 'income_withdrawal_savings',
       'income_inheritance_dowrie_legacies',
       'income_trademarks_patents_copyrights'],
      dtype='object')

In [185]:
expenditures.columns

Index(['hhid', 'expense_credit_card_payments',
       'expense_debt_to_work_or_person', 'expense_deposits_savings',
       'expense_foreigncurrencies_perciousmetals_art',
       'expense_hhbusiness_machinery_animal_equipment',
       'expense_hhbusiness_negative_balance', 'expense_life_insurance',
       'expense_loans_to_third_parties', 'expense_other',
       'expense_realestate_hh_inhabit', 'expense_realestate_hh_not_inhabit',
       'expense_realestate_mortgage', 'expense_securities_shares_bond',
       'expense_trademarks_patents_copyrights',
       'expense_inheritance_dowrie_legacies'],
      dtype='object')

In [191]:
aux = pd.DataFrame()
by_decile = ['expense_deposits_savings', 'income_withdrawal_savings', 'expense_credit_card_payments']
by_decile = ['income_house_land_condiminum']
for x in by_decile:
    for year in df.Y.unique():
        aux[x + str(int(year))] = utils.indicator_by_decile(df.query('Y == @year'), 'log_income', x, 'weight')

In [192]:
aux

,income_house_land_condiminum1994,income_house_land_condiminum1996,income_house_land_condiminum1998,income_house_land_condiminum2000
0,0.023863,0.002879,0.021919,0.016283
1,0.007455,0.003267,0.020763,0.011306
2,0.007173,0.004985,0.008234,0.017629
3,0.000633,0.003571,0.019042,0.02689
4,0.007155,0.0,0.01295,0.022333
5,0.003649,0.003497,0.011147,0.010149
6,0.011641,0.007935,0.041404,0.025393
7,0.00615,0.0,0.075162,0.038708
8,0.056193,0.00841,0.047462,0.027088
9,0.048857,0.005196,0.037797,0.039262
